### Caeden Kropf
### College Attendance Project
### 12/12/25

In [15]:
import requests
from bs4 import BeautifulSoup
import time, random
import pandas as pd
import time, random
import re
from IPython.display import HTML
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from urllib.request import urlopen
from json import loads

### CFB Stats Scraping

##### Getting Valid Team Ids from CFB Stats for scraping

In [ ]:
base_url = "https://cfbstats.com/2021/team"
index_url = f"{base_url}/index.html"

print("Starting scrape: conferences, teams, and average home attendance...\n")

# Step 1: Get all teams + their conferences from the index page
resp = requests.get(index_url)
soup = BeautifulSoup(resp.text, "html.parser")

conference_data = []

for conf_div in soup.find_all("div", class_="conference"):
    conf_name = conf_div.find("h1").text.strip()

    for team_link in conf_div.find_all("a"):
        team_name = team_link.text.strip()
        href = team_link["href"]  # e.g. /2024/team/721/index.html
        team_id = href.split("/")[3]  # Extract numeric team ID
        conference_data.append({
            "team_id": int(team_id),
            "team_name": team_name,
            "conference": conf_name
        })

conference_df = pd.DataFrame(conference_data)
print(f"Found {len(conference_df)} teams across {conference_df['conference'].nunique()} conferences.\n")

# Step 2: Loop through each team to get their average home attendance + record
attendance_records = []

for _, row in conference_df.iterrows():
    team_id = row["team_id"]
    team_name = row["team_name"]
    conference = row["conference"]

    team_url = f"{base_url}/{team_id}/index.html"
    print(f"Scraping {team_name} ({conference}) → {team_url}")

    try:
        resp = requests.get(team_url, timeout=10)
        if resp.status_code != 200:
            print(f"Skipping {team_name} (status {resp.status_code})")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")

        # ---- A) Extract attendance from schedule table (same as before) ----
        rows = soup.select("table tr")
        home_attendances = []

        for r in rows:
            cols = [c.text.strip() for c in r.find_all("td")]
            if len(cols) >= 5:
                date, opponent, result, game_time, attendance = cols[:5]

                # Clean attendance
                attendance = attendance.replace(",", "").strip()
                attendance = int(attendance) if attendance.isdigit() else None

                # Keep home games only (no '@' or '+')
                if not opponent.startswith("@") and not opponent.startswith("+") and attendance:
                    home_attendances.append(attendance)

        avg_home = int(round(sum(home_attendances) / len(home_attendances))) if home_attendances else None

        # ---- B) Extract record from "Team Record" table (All Games row) ----
        wins, losses = None, None

        for table in soup.find_all("table"):
            found = False
            for r in table.find_all("tr"):
                cells = r.find_all("td")
                if len(cells) >= 2:
                    split_label = cells[0].get_text(strip=True)
                    if split_label == "All Games":
                        record_text = cells[1].get_text(strip=True)  # e.g. "8-4"
                        # Clean and split "8-4" or "8 - 4"
                        record_text = record_text.replace(" ", "")
                        parts = record_text.split("-")

                        if len(parts) >= 2 and parts[0].isdigit() and parts[1].isdigit():
                            wins = int(parts[0])
                            losses = int(parts[1])
                        found = True
                        break
            if found:
                break

        if wins is not None and losses is not None and (wins + losses) > 0:
            win_pct = wins / (wins + losses)
        else:
            win_pct = None

        attendance_records.append({
            "team_id": team_id,
            "team_name": team_name,
            "conference": conference,
            "wins": wins,
            "losses": losses,
            "win_pct": round(win_pct, 3) if win_pct is not None else None,
            "avg_home_attendance": avg_home,
            "num_home_games": len(home_attendances)
        })

        print(f"{team_name}: {wins}-{losses} ({round(win_pct,3) if win_pct else 'N/A'}) | "
              f"{avg_home if avg_home is not None else 'N/A'} avg over {len(home_attendances)} home games\n")

    except Exception as e:
        print(f"Error scraping {team_name}: {e}")

    time.sleep(random.uniform(2, 5))  # polite delay

# Step 3: Combine results and save
attendance_df = pd.DataFrame(attendance_records)
attendance_df = attendance_df.sort_values("avg_home_attendance", ascending=False)
attendance_df.to_csv("cfbstats_conference_attendance.csv", index=False)

print("\nScraping complete! Saved as 'cfbstats_conference_attendance.csv'")
print(f"Total teams with data: {len(attendance_df)}")


Starting scrape: conferences, teams, and average home attendance...

Found 130 teams across 11 conferences.

Scraping Cincinnati (American Athletic Conference) → https://cfbstats.com/2021/team/140/index.html
Cincinnati: 13-1 (0.929) | 37338 avg over 7 home games

Scraping East Carolina (American Athletic Conference) → https://cfbstats.com/2021/team/196/index.html
East Carolina: 7-5 (0.583) | 36059 avg over 6 home games

Scraping Houston (American Athletic Conference) → https://cfbstats.com/2021/team/288/index.html
Houston: 12-2 (0.857) | 25073 avg over 5 home games

Scraping Memphis (American Athletic Conference) → https://cfbstats.com/2021/team/404/index.html
Memphis: 6-6 (0.5) | 31295 avg over 7 home games

Scraping Navy (American Athletic Conference) → https://cfbstats.com/2021/team/726/index.html
Navy: 4-8 (0.333) | 31094 avg over 6 home games

Scraping SMU (American Athletic Conference) → https://cfbstats.com/2021/team/663/index.html
SMU: 8-4 (0.667) | 23171 avg over 6 home games


##### Remapping Conference Names

In [83]:
#Loads existing dataset that was just scraped
attendance_df = pd.read_csv("cfbstats_conference_attendance.csv")

#Cleans up conference names
attendance_df["conference"] = attendance_df["conference"].replace({
    "Big Ten Conference": "Big Ten",
    "Southeastern Conference": "SEC",
    "Pac-12 Conference": "Pac-12",
    "Atlantic Coast Conference": "ACC",
    "Big 12 Conference": "Big 12",
    "American Athletic Conference": "AAC",
    "Conference USA": "C-USA",
    "Mid-American Conference": "MAC",
    "Mountain West Conference": "Mountain West",
    "Sun Belt Conference": "Sun Belt",
    "FBS Independents": "Independent"
})

display(attendance_df)

,team_id,team_name,conference,wins,losses,win_pct,avg_home_attendance,num_home_games
0,418,Michigan,Big Ten,12,2,0.857,108763.0,7
1,539,Penn State,Big Ten,7,6,0.538,106799.0,7
2,697,Texas A&M,SEC,8,4,0.667,102883.0,7
3,8,Alabama,SEC,13,2,0.867,98720.0,7
4,518,Ohio State,Big Ten,11,2,0.846,96756.0,7
...,...,...,...,...,...,...,...,...
125,5,Akron,MAC,2,10,0.167,10194.0,6
126,503,Northern Illinois,MAC,9,5,0.643,9058.0,6
127,400,Massachusetts,Independent,1,11,0.083,8994.0,6
128,277,Hawaii,Mountain West,6,7,0.462,4605.0,3


In [85]:
#Loads the CFBD metadata file
cfbd_df = pd.read_csv("cfbd.csv")

#Renames some columns
cfbd_df.rename(columns={'School': 'team_name','Location City':'city', 'Id':'Logo Id', 'Location Name':'stadium'}, inplace=True)

#Looks for teams that were scraped that are not in the CFBD set due to different names
missing_in_cfbd = sorted(set(attendance_df["team_name"].unique()) - set(cfbd_df["team_name"].unique()))
if missing_in_cfbd:
    print("Teams in attendance_df not found in cfbd_df:")
    print(missing_in_cfbd)

Teams in attendance_df not found in cfbd_df:
['Appalachian State', 'Connecticut', 'FIU', 'Hawaii', 'Louisiana-Monroe', 'Miami (FL)', 'Middle Tennessee State', 'San Jose State', 'USF']


In [87]:
#Fix common mismatches so merge works
cfbd_df["team_name"] = cfbd_df["team_name"].replace({
    "App State": "Appalachian State",
    "UConn": "Connecticut",
    "Florida International": "FIU",
    "Hawai'i": "Hawaii",
    "UL Monroe": "Louisiana-Monroe",
    "Miami": "Miami (FL)",
    "Middle Tennessee": "Middle Tennessee State",
    "San José State": "San Jose State",
    "South Florida": "USF"
})

#Double checks for any mismatches
missing_in_cfbd = sorted(set(attendance_df["team_name"].unique()) - set(cfbd_df["team_name"].unique()))
if missing_in_cfbd:
    print("Teams in attendance_df not found in cfbd_df:")
    print(missing_in_cfbd)
else:
    print("All team names match")

All team names match


In [ ]:
#Picks columns to keep from CFBD dataset
cfbd_columns_to_keep = [
    "team_name",    
    "Logo Id",                  
    "Color",           
    "AlternateColor",          
    "stadium",
    "city",
    "Location State",
    "Location Latitude",
    "Location Longitude",
    "Location Capacity"
]

#Keeps only those columns (ignore missing ones safely)
cfbd_trimmed = cfbd_df.loc[:, [col for col in cfbd_columns_to_keep if col in cfbd_df.columns]].copy()

#Perform the merge, only keeping teams that appear in attendance_df
merged_df = pd.merge(
    attendance_df,
    cfbd_trimmed,
    on="team_name",
    how="inner"
)


display(merged_df)
merged_df.to_csv("cfbstats_cfbd_merged.csv", index=False)
print("\nSaved as 'cfbstats_cfbd_merged.csv'")

,team_id,team_name,conference,wins,losses,win_pct,avg_home_attendance,num_home_games,Logo Id,Color,AlternateColor,city,Location State,Location Latitude,Location Longitude,Location Capacity
0,418,Michigan,Big Ten,12,2,0.857,108763.0,7,130,#00274c,#ffcb05,Ann Arbor,MI,42.265836,-83.748696,107601
1,539,Penn State,Big Ten,7,6,0.538,106799.0,7,213,#041E42,#FFFFFF,University Park,PA,40.812196,-77.856102,106572
2,697,Texas A&M,SEC,8,4,0.667,102883.0,7,245,#500000,#ffffff,College Station,TX,30.609889,-96.340383,102733
3,8,Alabama,SEC,13,2,0.867,98720.0,7,333,#9e1632,#ffffff,Tuscaloosa,AL,33.208275,-87.550384,101821
4,518,Ohio State,Big Ten,11,2,0.846,96756.0,7,194,#ce1141,#505056,Columbus,OH,40.001645,-83.019727,102780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,5,Akron,MAC,2,10,0.167,10194.0,6,2006,#00285e,#84754e,Akron,OH,41.072553,-81.508341,30000
126,503,Northern Illinois,MAC,9,5,0.643,9058.0,6,2459,#F1122C,#cc0000,DeKalb,IL,41.933959,-88.777836,23595
127,400,Massachusetts,Independent,1,11,0.083,8994.0,6,113,#971B2F,#B1B3B3,Hadley,MA,42.377324,-72.536059,17000
128,277,Hawaii,Mountain West,6,7,0.462,4605.0,3,62,#003420,#ffffff,Honolulu,HI,21.294294,-157.819338,9000



Saved as 'cfb_attendance_with_locations.csv'


### Educational Data API Scraping

In [92]:
#Reads url and gets results from IPEDS directory
url = "https://educationdata.urban.org/api/v1/college-university/ipeds/directory/2021/"
response = urlopen(url)
data = loads(response.read())
print("Keys:", data.keys())
print("Number of schools:", len(data["results"]))

#Puts results into dataframe
df_direct = pd.DataFrame(data["results"])

#Keeps specific columns
keep_cols = ["unitid", "inst_name", "state_abbr", "city", "county_name"]
df_direct = df_direct[keep_cols]

df_direct.head()

Keys: dict_keys(['count', 'next', 'previous', 'results'])
Number of schools: 6289


,unitid,inst_name,state_abbr,city,county_name
0,100654,Alabama A & M University,AL,Normal,Madison County
1,100663,University of Alabama at Birmingham,AL,Birmingham,Jefferson County
2,100690,Amridge University,AL,Montgomery,Montgomery County
3,100706,University of Alabama in Huntsville,AL,Huntsville,Madison County
4,100724,Alabama State University,AL,Montgomery,Montgomery County


In [94]:
#Reads url and gets results from IPEDS institutional characterisitcs
url = "https://educationdata.urban.org/api/v1/college-university/eada/institutional-characteristics/2021/"
response = urlopen(url)
data = loads(response.read())
print("Keys:", data.keys())
print("Number of schools:", len(data["results"]))

#Puts results into dataframe
df_ath = pd.DataFrame(data["results"])

#Keeps specific columns
keep_cols = ["unitid", "enrollment_men", "enrollment_women", "enrollment_total", "ath_classification_code"]
df_ath = df_ath[keep_cols]

df_ath.head()

Keys: dict_keys(['count', 'next', 'previous', 'results'])
Number of schools: 2028


,unitid,enrollment_men,enrollment_women,enrollment_total,ath_classification_code
0,800001,5723,6570,12293,4
1,100654,1823,2706,4529,2
2,100663,3924,6442,10366,1
3,100706,3726,2491,6217,5
4,100724,1053,2002,3055,2


In [96]:
merged = (
    df_direct
    .merge(df_ath, on="unitid", how="left")
)

merged = merged.drop(columns=["unitid"])

merged.head()

,inst_name,state_abbr,city,county_name,enrollment_men,enrollment_women,enrollment_total,ath_classification_code
0,Alabama A & M University,AL,Normal,Madison County,1823.0,2706.0,4529.0,2.0
1,University of Alabama at Birmingham,AL,Birmingham,Jefferson County,3924.0,6442.0,10366.0,1.0
2,Amridge University,AL,Montgomery,Montgomery County,NaN,NaN,NaN,NaN
3,University of Alabama in Huntsville,AL,Huntsville,Madison County,3726.0,2491.0,6217.0,5.0
4,Alabama State University,AL,Montgomery,Montgomery County,1053.0,2002.0,3055.0,2.0


In [98]:
merged.to_csv("ipeds_directory_plus_enroll.csv", index=False)
print("Saved: ipeds_directory_plus_enroll.csv")

Saved: ipeds_directory_plus_enroll.csv


### Merge CFBStats and Educational API

In [ ]:
#Reads in CSV files
cfb = pd.read_csv("cfbstats_cfbd_merged.csv")
ipeds = pd.read_csv("ipeds_directory_plus_enroll.csv")

In [103]:
#Function that cleans both city columns to make sure they match
def clean_city(x):
    return (
        str(x).lower()
              .replace(".", "")
              .replace(",", "")
              .strip()
    )

#Applies the function to both city columns
cfb["city_clean"] = cfb["city"].apply(clean_city)
ipeds["city_clean"] = ipeds["city"].apply(clean_city)

#Filters down schools who participate in D1 athletics to make merging easier
ipeds_d1 = ipeds[ipeds["ath_classification_code"] == 1]

In [105]:
#Merges both datasets on the 'city_clean' column
merged_city = pd.merge(
    cfb,
    ipeds_d1,
    on="city_clean",
    how="left"
)

#Drops the 'city_clean' column
merged_city.drop(columns=["city_clean"], inplace=True)

#Sorts the data frame
merged_city_sorted = merged_city.sort_values(
    by=["team_name", "enrollment_total"], 
    ascending=[True, False]
)

#Drops duplicate team_name
merged_on_city = merged_city_sorted.drop_duplicates(subset=["team_name"], keep="first")

print(merged_on_city.shape)
merged_on_city.head()

(130, 24)


,team_id,team_name,conference,wins,losses,win_pct,avg_home_attendance,num_home_games,Logo Id,Color,...,Location Longitude,Location Capacity,inst_name,state_abbr,city_y,county_name,enrollment_men,enrollment_women,enrollment_total,ath_classification_code
88,721,Air Force,Mountain West,10,3,0.769,22831.0,6,2005,#004a7b,...,-104.843616,46692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,5,Akron,MAC,2,10,0.167,10194.0,6,2006,#00285e,...,-81.508341,30000,University of Akron Main Campus,OH,Akron,Summit County,4940.0,4327.0,9267.0,1.0
3,8,Alabama,SEC,13,2,0.867,98720.0,7,333,#9e1632,...,-87.550384,101821,The University of Alabama,AL,Tuscaloosa,Tuscaloosa County,12366.0,15708.0,28074.0,1.0
74,27,Appalachian State,Sun Belt,10,4,0.714,29389.0,6,2026,#ffcc00,...,-81.685428,30000,Appalachian State University,NC,Boone,Watauga County,7505.0,9876.0,17381.0,1.0
64,29,Arizona,Pac-12,1,11,0.083,34900.0,6,12,#0c234b,...,-110.948868,50782,University of Arizona,AZ,Tucson,Pima County,12749.0,16691.0,29440.0,1.0


In [107]:
#Changes names of problem schools to match
ipeds_to_cfb = {
    "United States Air Force Academy": "Air Force",
    "United States Military Academy": "Army",
    "United States Naval Academy": "Navy",
    "University of Connecticut": "Connecticut",
    "University of Miami": "Miami (FL)",
    "Mississippi State University": "Mississippi State",
    "Rice University": "Rice",
    "University of California-Los Angeles": "UCLA",
    "University of Southern California": "USC",
    "University at Buffalo": "Buffalo",
    "Georgia Institute of Technology-Main Campus": "Georgia Tech",
    "University of Mississippi": "Ole Miss",
    "Rutgers University-New Brunswick": "Rutgers",
    "Southern Methodist University": "SMU"
}
ipeds_d1["team_name"] = ipeds_d1["inst_name"].replace(ipeds_to_cfb)

C:\Users\caede\AppData\Local\Temp\ipykernel_29628\2271561773.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipeds_d1["team_name"] = ipeds_d1["inst_name"].replace(ipeds_to_cfb)


In [109]:
#Creates a new list of rows that did not merge properly
problem_list = list(ipeds_to_cfb.values())

matched = merged_on_city[~merged_on_city["team_name"].isin(problem_list)]
ipeds_for_fix = ipeds_d1[ipeds_d1["team_name"].isin(problem_list)]
cfb_for_fix = cfb[cfb["team_name"].isin(problem_list)]

In [119]:
#Merges rows that used different cities or share a city with another school on other features
fixed_merge = pd.merge(
    cfb_for_fix,
    ipeds_for_fix,
    on="team_name",
    how="left"
)

final_merged = pd.concat([matched, fixed_merge], ignore_index=True)
final_merged = final_merged.drop_duplicates(subset=["team_name"], keep="first")

#Drops unnecessary and extra columns created during joins
final_merged.drop(columns=["inst_name", "city_clean", "Location State", "county_name", "city_y", "city_clean_x", "city_clean_y", "ath_classification_code", "team_id"], errors="ignore", inplace=True)

#Rename city_x to city
if "city_x" in final_merged.columns:
    final_merged.rename(columns={"city_x": "city"}, inplace=True)

#Drop schools with no information
drop_list = [
    "Army", "Navy", "Air Force", 
    "Massachusetts"
]
final_merged = final_merged[~final_merged["team_name"].isin(drop_list)]

In [121]:
#Converts all column names to snake case
final_merged.columns = (
    final_merged.columns.str.strip()              
                      .str.lower()      
                      .str.replace(" ", "_")  
                      .str.replace("-", "_") 
                      .str.replace("/", "_")  
                      .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
)


In [123]:
#Final dataset
final_merged.to_csv("final_college_cleaned.csv", index=False)
print("Saved: final_college_cleaned.csv")

Saved: final_college_cleaned.csv
